In [65]:
import pandas as pd
import miceforest as mf
from sklearn.linear_model import LinearRegression

In [66]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

In [67]:
df_train['Cabin_Type'] = df_train.loc[df_train['Cabin'].notna()]['Cabin'].apply(lambda x: x[0])
df_test['Cabin_Type'] = df_test.loc[df_test['Cabin'].notna()]['Cabin'].apply(lambda x: x[0])

df_train['Cabin_Nos'] = df_train.loc[df_train['Cabin'].notna()]['Cabin'].apply(lambda x: len(x.split()))
df_test['Cabin_Nos'] = df_test.loc[df_test['Cabin'].notna()]['Cabin'].apply(lambda x: len(x.split()))

df_train = df_train.drop(columns='Cabin')
df_test = df_test.drop(columns='Cabin')

# new_df_train = pd.get_dummies(df_train['Sex'])
# df_train = pd.concat([df_train, new_df_train], axis=1)

# new_df_test = pd.get_dummies(df_test['Sex'])
# df_test = pd.concat([df_test, new_df_test], axis=1)

# df_train = df_train.drop(columns=['Sex'])
# df_test = df_test.drop(columns=['Sex'])

df_train['Sex'] = df_train['Sex'].apply(lambda x: 1 if x=='male' else 0)
df_test['Sex'] = df_test['Sex'].apply(lambda x: 1 if x=='male' else 0)

df_train['Embarked'] = df_train['Embarked'].fillna('S')
df_test['Embarked'] = df_test['Embarked'].fillna('S')

new_df_train = pd.get_dummies(df_train['Embarked'])
new_cols = ['Embarked_{}'.format(x) for x in new_df_train.columns.values]
new_df_train.columns = new_cols
df_train = pd.concat([df_train, new_df_train], axis=1)

new_df_test = pd.get_dummies(df_test['Embarked'])
new_cols = ['Embarked_{}'.format(x) for x in new_df_test.columns.values]
new_df_test.columns = new_cols
df_test = pd.concat([df_test, new_df_test], axis=1)

df_train = df_train.drop(columns=['Embarked'])
df_test = df_test.drop(columns=['Embarked'])

new_df_train = pd.get_dummies(df_train['Cabin_Type'])
new_cols = ['Cabin_Type_{}'.format(x) for x in new_df_train.columns.values]
new_df_train.columns = new_cols
df_train = pd.concat([df_train, new_df_train], axis=1)

new_df_test = pd.get_dummies(df_test['Cabin_Type'])
new_cols = ['Cabin_Type_{}'.format(x) for x in new_df_test.columns.values]
new_df_test.columns = new_cols
df_test = pd.concat([df_test, new_df_test], axis=1)

df_train = df_train.drop(columns=['Cabin_Type'])
df_test = df_test.drop(columns=['Cabin_Type'])

new_df_train = pd.get_dummies(df_train['Pclass'])
new_cols = ['Pclass_{}'.format(x) for x in new_df_train.columns.values]
new_df_train.columns = new_cols
df_train = pd.concat([df_train, new_df_train], axis=1)

new_df_test = pd.get_dummies(df_test['Pclass'])
new_cols = ['Pclass_{}'.format(x) for x in new_df_test.columns.values]
new_df_test.columns = new_cols
df_test = pd.concat([df_test, new_df_test], axis=1)

df_train = df_train.drop(columns=['Pclass'])
df_test = df_test.drop(columns=['Pclass'])

df_train['Cabin_Nos'] = df_train['Cabin_Nos'].fillna(1)
df_test['Cabin_Nos'] = df_test['Cabin_Nos'].fillna(1)

df_test['Cabin_Type_T'] = 0

In [68]:
inp_col_list = list(df_train.columns.values)
inp_col_list.remove('PassengerId')
inp_col_list.remove('Survived')
inp_col_list.remove('Name')
inp_col_list.remove('Ticket')

X_train = df_train[inp_col_list]

inp_col_list = list(df_test.columns.values)
inp_col_list.remove('PassengerId')
inp_col_list.remove('Name')
inp_col_list.remove('Ticket')

X_test = df_test[inp_col_list]

In [69]:
df_train_amp = mf.ampute_data(X_train,perc=0.25,random_state=1991)
df_test_amp = mf.ampute_data(X_test,perc=0.25,random_state=1991)

In [70]:
# Create kernels. 
kernel_train = mf.ImputationKernel(
    data=df_train_amp,
    save_all_iterations=True,
    random_state=1991
)

# Run the MICE algorithm for 3 iterations on each of the datasets
kernel_train.mice(3)

In [71]:
# Create kernels. 
kernel_test = mf.ImputationKernel(
    data=df_test_amp,
    save_all_iterations=True,
    random_state=1991
)

# Run the MICE algorithm for 3 iterations on each of the datasets
kernel_test.mice(3)

In [72]:
df_train_imp = kernel_train.impute_new_data(df_train_amp).complete_data(0)
df_test_imp = kernel_test.impute_new_data(df_test_amp).complete_data(0)

In [73]:
df_train_imp = pd.concat([df_train[['PassengerId', 'Survived', 'Name', 'Ticket']], df_train_imp], axis=1)
df_test_imp = pd.concat([df_test[['PassengerId', 'Name', 'Ticket']], df_test_imp], axis=1)

In [74]:
df_train_imp.to_csv('../data/train_clean_imp.csv', index=False)
df_test_imp.to_csv('../data/test_clean_imp.csv', index=False)

In [76]:
df_train_imp.isnull().sum()

PassengerId     0
Survived        0
Name            0
Ticket          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            0
Cabin_Nos       0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
Cabin_Type_A    0
Cabin_Type_B    0
Cabin_Type_C    0
Cabin_Type_D    0
Cabin_Type_E    0
Cabin_Type_F    0
Cabin_Type_G    0
Cabin_Type_T    0
Pclass_1        0
Pclass_2        0
Pclass_3        0
dtype: int64